In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title cd
%pwd
%cd /content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet

/content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet


In [3]:
#@title install packages
%pip install chainer
%pip install -Uqq ipdb

     |████████████████████████████████| 1.0 MB 9.9 MB/s 
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967740 sha256=bdc7889fa3a6c2411599a011ecffad97380c91a5f0be77dac8ffb3f6e37eae77
  Stored in directory: /root/.cache/pip/wheels/c8/6a/6f/fd563166cc597e5206e375ea074ea836e5db5dd58421215672
Successfully built chainer
     |████████████████████████████████| 793 kB 9.4 MB/s 
     |████████████████████████████████| 381 kB 78.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.33.0 which is incompatible.


In [4]:
#@title read gpu details
import chainer
#chainer.print_runtime_info()

print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


GPU availability: True
cuDNN availablility: True
Fri May  6 22:02:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                       

In [ ]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
#import torch_xla
#import torch_xla.core.xla_model as xm

In [5]:
#@title imports
"""
high level support for doing this and that.
"""
from __future__ import print_function
import time
import csv
import numpy as np
import torch
import torch.nn.functional as F
from scipy import stats
from init import initializer
from eenet import EENet
from custom_eenet import CustomEENet
from gatednet import GatedNet
import matplotlib.pylab as plt
import loss_functions
import utils
import config
import gc
import sys
import math
import os
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms.functional as tf
import matplotlib.pyplot as plt
import ipdb

import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import joblib as jobl
from collections import Counter
from matplotlib import pyplot
from numpy import where
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor

%matplotlib inline


In [ ]:
#@title add noise
def add_noise(args, image):
    mean_i = (0.4914, 0.4822, 0.4465)
    std_i = (0.2023, 0.1994, 0.2010)
    #ipdb.set_trace(context=6)

    for t, m, s in zip(image, mean_i, std_i):
        t.mul_(s).add_(m)
    
    """
%matplotlib inline

    # Generate the noise as you did
    im = image[0].detach().cpu().numpy()
    #ipdb.set_trace(context=6)
    im = np.transpose(im,(1,2,0))
    plt.imshow(im) 
    plt.show()
    #ipdb.set_trace(context=6)
    """

    _, ch, row, col = image.shape
    mean_n = 0
    var_n = 0.1
    sigma = var_n**0.5
    noise = torch.normal(mean_n,sigma,(1, ch, row, col))
    snr = 10.0 ** (args.noise_snr / 10.0)
    #ipdb.set_trace(context=6)
    # work out the current SNR
    current_snr = torch.mean(image) / torch.std(noise)

    # scale the noise by the snr ratios (smaller noise <=> larger snr)
    noise = noise * (current_snr / snr)
    img_noise = image + noise
    
    """
    im = img_noise[0].detach().cpu().numpy()
    im = np.transpose(im,(1,2,0))
    plt.imshow(im) 
    plt.show()
    ipdb.set_trace(context=6)
    """

    #utils.save_image(args, img_noise, 'image1')
    img_noise = tf.normalize(img_noise, mean_i, std_i)

    # return the new signal with noise
    return img_noise

In [ ]:
#@title plot_ee_histogram
def plot_ee_histogram(args, val_confs, type_loader, dir):
    %matplotlib inline
    x_pos = np.arange(args.num_ee+1)
    #print(x_pos)
    print(val_confs)
    bar = plt.bar(x_pos, val_confs, align='center')
    plt.xlabel('Exit number')
    plt.ylabel('# of Samples in Exit')
    plt.title(args.dataset + ' samples distribution in exits using ' + args.model + 
              ' model\n' + 'SNR=' + str(args.noise_snr) + 'dB')
    plt.ylim(0, max(val_confs))
    plt.savefig(dir + '/ee_hist_' + type_loader + '.' + str(args.loss_threshold) + '.png')
    plt.show()
    

In [ ]:
#@title plot_noise_exits
import plotly.graph_objects as go
import matplotlib

#@title plot_noise_exits
def plot_noise_exits_box(args, type_loader):
    %matplotlib inline
    x_pos = np.arange(args.num_noise_levels)

    for thresh in range(1, 13, 1):
        loss_threshold = thresh/10
        data_file = args.relative_losses_dir + '/noise_exits_' + type_loader + '.' + str(loss_threshold) + '.pt'
        noise_exits = torch.load(data_file)
        #noise_exits = np.array(noise_exits)

        #ipdb.set_trace(context=6)
        mean=np.mean(noise_exits,axis=1)
        print(mean)

        fig, ax = plt.subplots()#figure(figsize =(10, 7))
    
        # Creating axes instance
        ax.set_title(args.dataset + ' samples distribution in exits using ' + args.model + ' with threshold=' + str(loss_threshold))
        ax.set_xlabel('noise index')
        ax.set_ylabel('# exits')
        
        # Creating plot
        bp = ax.boxplot(noise_exits)
        
        # show plot
        plt.show()
        plt.savefig(args.relative_losses_dir + '/bp_noises_exits_' + '.' + str(loss_threshold) + '.png')
        
def plot_noise_exits_scatter(args, type_loader):
    %matplotlib inline
    #x_pos = np.arange(args.num_noise_levels)
    noise_levels = np.arange(args.min_noise_snr, args.max_noise_snr,
        (args.max_noise_snr - args.min_noise_snr)/args.num_noise_levels) 

    x = noise_levels
    selected_losses = np.array([0.3, 0.5, 0.7, 0.9])
    colors = ['green', 'black', 'red', 'blue']
    fig = go.Figure()

    #for thresh in selected_losses:
    for idx, thresh in enumerate(selected_losses):
        loss_threshold = thresh#/10
        data_file = args.relative_losses_dir + '/noise_exits_' + type_loader + '.' + str(loss_threshold) + '.pt'
        data = torch.load(data_file)
        #noises_data.append(data)

        #mean=np.median(data,axis=1)
        mean=np.mean(data,axis=1)
        std=np.std(data,axis=1)
        q1=np.quantile(data,q=0.25,axis=1)
        q3=np.quantile(data,q=0.75,axis=1)
        """
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(x=x, y=q3,
                                mode='lines',
                                line=dict(color='red',width =0.1),
                                name='upper bound'))
        """
        fig.add_trace(go.Scatter(x=x, y=mean,
                                mode='lines',
                                line=dict(color=colors[idx]),
                                #fill='tonexty',
                                name='threshold ' + str(loss_threshold)))
        """
        fig.add_trace(go.Scatter(x=x, y=q1,
                                mode='lines',
                                line=dict(color='blue', width =0.1),
                                fill='tonexty',
                                name='lower bound'))
        """
        #ipdb.set_trace(context=6)
    fig.update_layout(
        title=args.dataset + ' samples distribution in exits using ' + args.model,# + ' with threshold=' + str(loss_threshold),
        xaxis_title="noise level in SNR (dB)",
        yaxis_title="# exit",
        xaxis = dict(
            tickmode = 'array',
            tickvals = noise_levels,
        ),
        width=700,
        height=500)

    #plt.show()
    fig.show()
    plt.savefig(args.relative_losses_dir + '/bp_noises_exits_' + '.' + str(args.loss_threshold) + '.png')

In [ ]:
#@title exit_flow_analysis
def exit_flow_analysis(args, model, val_loader):
    """examine the model output.
    Arguments are
    * args:         command line arguments entered by user.
    * model:        convolutional neural network model.
    * train_loader: train data loader.
    This examines the outputs of already trained model.
    """
    model.eval()
    val_confs_true = []
    val_confs_false = []
    vals_len_true = np.zeros(args.num_ee+1)
    vals_len_false = np.zeros(args.num_ee+1)
    for i in range(args.num_ee+1):
        val_confs_true.append([])
        val_confs_false.append([])
    
    #val_confs[0].append('aa2')
    #val_confs = np.empty((args.num_ee, 0), float)#np.zeros(len(val_loader.dataset))
    #i = 0
    """
    print(args.results_dir+'/pred_vs_conf.csv')
    experiment = open(args.results_dir+'/pred_vs_conf.csv', 'w', newline='')
    recorder = csv.writer(experiment, delimiter=',')
    recorder.writerow(['target',
                       'start_pred_seq',
                       'start_conf_seq',
                       'start_exit_seq',
                       'actual_pred',
                       'actual_conf',
                       'actual_exit'])
    """
    
    print('add noise w/ SNR=' + str(args.testing_snr) + 'dB')
    
    with torch.no_grad():
        for data, target in val_loader:
            data = add_noise(args, data)
            
            data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
            pred, conf, cost, idx = model(data)
            loss = F.nll_loss(pred.log(), target)

            pred = pred.max(1, keepdim=True)[1].item()
            conf = conf.item()
            #print(loss)
            #conf = torch.max(confs).item()
            #i += 1
            #confs = [c.item() for c in confs]
            loss = loss.item()
            target = target.item()
            if target == pred:
                val_confs_true[idx].append(conf)
            else:
                val_confs_false[idx].append(conf)

            #val_confs[idx].append(conf)

    total_correct = 0
    for i in range(args.num_ee+1):
        total_correct += len(val_confs_true[i])
        if len(val_confs_true[i]) > 1:
            filename = 'test_true_confs_hist_ee' + str(i)
            vals_len_true[i] = len(val_confs_true[i])
            utils.plot_histogram(args, val_confs_true[i], 'conf', 100, filename, saveplot=True)
        if len(val_confs_false[i]) > 1:
            filename = 'test_false_confs_hist_ee' + str(i)
            vals_len_false[i] = len(val_confs_false[i])
            utils.plot_histogram(args, val_confs_false[i], 'conf', 100, filename, saveplot=True)
    flat_list_true = [item for sublist in val_confs_true for item in sublist]
    flat_list_false = [item for sublist in val_confs_false for item in sublist]
    
    utils.plot_histogram(args, flat_list_true, 'conf true', 100, 'global_conf_true', saveplot=True)
    utils.plot_histogram(args, flat_list_false, 'conf false', 100, 'global_conf_false', saveplot=True)
    print('total currect: ', total_correct)
    print('total acc : ', total_correct * 100 /10000)
    plot_ee_histogram(args, vals_len_true)
    plot_ee_histogram(args, vals_len_false)
    return

In [6]:
#@title Train
def train(args, model, train_loader, optimizer, exit_tags):
    """train the model.

    Arguments are
    * args:         command line arguments entered by user.
    * model:        convolutional neural network model.
    * train_loader: train data loader.
    * optimizer:    optimize the model during training.
    * epoch:        epoch number.

    This trains the model and prints the results of each epochs.
    """
    losses = []
    pred_losses = []
    cost_losses = []
    model.train()
    
    # actual training starts
    #for batch_id, ((data, noise), target) in enumerate(train_loader):
    for batch_id, (data, target, _) in enumerate(train_loader):
        # add noise to input data
        #ipdb.set_trace(context=6)
        #if args.add_noise != 0:
        #    data = add_noise(args, data)
        # fetch the current batch data
        data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
        exit_tag = None
        if args.loss_func == 'v4':
            exit_tag = exit_tags[batch_id].to(args.device)
        optimizer.zero_grad()
        cum_loss = 0

        # training settings for EENet based models
        if isinstance(model, (CustomEENet, EENet)):
            pred, conf, cost, _ = model(data)
            cost.append(1)#torch.tensor(1.0).to(args.device))
            
            if args.use_main_targets:
                _, target = torch.max(pred[args.num_ee], 1)
                #cum_loss, pred_loss, cost_loss = loss_functions.loss(args, exit_tag, pred, target, conf, cost)
                         
            if config.mode == 'train_main':
                cum_loss = F.nll_loss(pred[args.num_ee].log(), target)
                #cum_loss = F.cross_entropy(pred[args.num_ee], target)
                pred_loss = cum_loss
                cost_loss = 1.0
            elif config.mode == 'train_ee':
                #ipdb.set_trace(context=6)                
                cum_loss, pred_loss, cost_loss = loss_functions.loss(args, exit_tag, pred, target, conf, cost)
            else:
                print('Error: undefined mode..')
                return        
                """
                if args.ee_disable:
                    cum_loss = F.nll_loss(pred[args.num_ee].log(), target)
                    #cum_loss = F.cross_entropy(pred[args.num_ee], target)
                    pred_loss = cum_loss
                    cost_loss = 1.0
                else:
                    cum_loss, pred_loss, cost_loss = loss_functions.loss(args, exit_tag, pred, target, conf, cost)
                    #cum_loss, loss = calc_loss(args, pred, conf, cost, target, exit_id)
                """

        # training settings for other models
        else:
            pred = model(data)
            cum_loss = F.cross_entropy(pred, target)

        losses.append(float(cum_loss))
        pred_losses.append(float(pred_loss))
        cost_losses.append(float(cost_loss))
        cum_loss.backward()
        optimizer.step()
        """
        for obj in gc.get_objects():
            try:
                if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                    print(type(obj), obj.size())
            except:
                pass
        """
        #print(model.exits[1].classifier[0].weight)

        # update the exit tags of inputs
    #print(pred_loss)
    #print('target')
    #print(target)
    if args.loss_func == 'v4' and args.ee_disable == False:
        for batch_id, (data, target) in enumerate(train_loader):
            data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
            batch_size = len(exit_tags[batch_id])
            pred, _, cost = model(data)

            if args.use_main_targets:
                _, target = torch.max(pred[args.num_ee], 1)
            exit_tags[batch_id] = loss_functions.update_exit_tags(args, batch_size,
                                                                  pred, target, cost)


    # print the training results of epoch
    result = {'train_loss': round(np.mean(losses), 4),
              'train_loss_sem': round(stats.sem(losses), 2),
              'pred_loss': round(np.mean(pred_losses), 4),
              'pred_loss_sem': round(stats.sem(pred_losses), 2),
              'cost_loss': round(np.mean(cost_losses), 4),
              'cost_loss_sem': round(stats.sem(cost_losses), 2)}

    print('Train avg loss: {:.4f}'.format(result['train_loss']))
    return result

In [ ]:
#@title train_gated_xgboost
"""
params={
'booster':'gbtree',
'objective': 'multi:softmax',
'num_class':11,
'gamma':0.6,#0.1,
'max_depth':10, #6
'lambda': 3,#2,
'alpha': 4, #new
'subsample':0.7,
'colsample_bytree':0.7,
'min_child_weight':1,
'silent':0 ,
'eta': 0.01, #0.01
'seed':1000,
'tree_method': 'gpu_hist',
'gpu_id': 0,
}
"""
depth = 8
params={
'booster':'gbtree',
'objective': 'multi:softmax',
'num_class':11,
'gamma':0.6,#0.1,
'max_depth':depth, #6
'lambda': 3,#2,
'alpha': 4, #new
'subsample':0.7,
'colsample_bytree':0.7,
'min_child_weight':1,
'silent':0 ,
'eta': 0.03, #0.01
'seed':1000,
'tree_method': 'gpu_hist',
'gpu_id': 0,
}

#82%

def evaluate(model, X, Y):#, weights):
    predicted_Y = model.predict(X)
    #ipdb.set_trace(context=6)
    print(predicted_Y)
    accuracy = accuracy_score(Y, predicted_Y)#, weights)
    return accuracy
"""
def produce_weights(data, size, num_of_classes):
    counter = Counter(data)
    weights = np.zeros(num_of_classes)
    weights_vec = np.zeros(size)
    for key in counter:
        #print(key)
        weights[int(key)] = counter[key]
    weights = 1 - weights / np.max(weights) + 0.001
    #ipdb.set_trace(context=6)
    #print(weights)
    for i, value in enumerate(data):
        weights_vec[i] = weights[value]
    return weights_vec
""" 

def train_gated_xgboost(args, noise_levels):
    log_file = open(args.gated_models_dir+'/train_gated_xgboost_log.txt', 'a', newline='')
    log_file.write("Training gated xgboost mode:\n")
    log_file.write("loss threshold: " + str(args.loss_threshold) + ", SNR=" + str(args.noise_snr) + "\n")
    
    %matplotlib inline

    type_loader = 'test'
    ground_truth_noisy_dir = args.ground_truths_dir + '/' + str(args.loss_threshold)# + '/' + 'snr_' + str(noise_levels[0])
    """
    data_file_train = ground_truth_noisy_dir + '/datas_train.' + str(args.loss_threshold) + '.pt'
    data_train = torch.load(data_file_train)
    print('loaded data train at: ', data_file_train)
    
    gt_file_train = ground_truth_noisy_dir + '/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_train = torch.load(gt_file_train)
    print('loaded exits ground truths train at: ', gt_file_train)
    """
    data_file_test = ground_truth_noisy_dir + '/datas_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    
    gt_file_test = ground_truth_noisy_dir + '/exit_idx_predictions_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_test = torch.load(gt_file_test)
    print('loaded exits ground truths test at: ', gt_file_test)
    
    if 0:
        for m in range(args.num_noise_levels-1):
            ground_truth_noisy_dir = args.ground_truths_dir + '/' + str(args.loss_threshold) + '/' + 'snr_' + str(noise_levels[m+1])
            """
            data_file_train = ground_truth_noisy_dir + '/datas_train.' + str(args.loss_threshold) + '.pt'
            data = torch.load(data_file_train)
            print('loaded data train at: ', data_file_train)
            data_train = np.concatenate((data_train, data), axis=0)

            gt_file_train = ground_truth_noisy_dir + '/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
            data = torch.load(gt_file_train)
            print('loaded exits ground truths train at: ', gt_file_train)
            exits_ground_truth_train = np.concatenate((exits_ground_truth_train, data), axis=0)
            """
            data_file_test = ground_truth_noisy_dir + '/datas_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
            data = torch.load(data_file_test)
            print('loaded data test at: ', data_file_test)
            data_test = np.concatenate((data_test, data), axis=0)

            gt_file_test = ground_truth_noisy_dir + '/exit_idx_predictions_' + type_loader + '.' + str(args.loss_threshold) + '.pt'
            data = torch.load(gt_file_test)
            print('loaded exits ground truths test at: ', gt_file_test)
            exits_ground_truth_test = np.concatenate((exits_ground_truth_test, data), axis=0)
        
    """
    ipdb.set_trace(context=6)
    gt_file_train = args.ground_truths_dir + '/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_train = torch.load(gt_file_train)
    print('loaded exits ground truths at: ', gt_file_train)
    train_len = len(exits_ground_truth_train)
    #print('exits_ground_truth_train {:8d}'.format(train_len))
    
    gt_file_test = args.ground_truths_dir + '/exit_idx_predictions_test.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_test = torch.load(gt_file_test)
    print('loaded exits ground truths at: ', gt_file_test)
    test_len = len(exits_ground_truth_test)
    #print('exits_ground_truth_test {:8d}'.format(test_len))
    
    data_file_train = args.ground_truths_dir + '/datas_train.' + str(args.loss_threshold) + '.pt'
    data_train = torch.load(data_file_train)
    print('loaded data train at: ', data_file_train)
    data_train_len = len(data_train)
    #print('data_train_len {:8d}'.format(data_train_len))
    
    data_file_test = args.ground_truths_dir + '/datas_test.' + str(args.loss_threshold) + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    data_test_len = len(data_test)
    #print('data_test_len {:8d}'.format(data_test_len))
    """
    #ipdb.set_trace(context=6)
    #data_train = torch.stack(data_train, dim=0)
    #data_test = torch.stack(data_test, dim=0)
    #d1, d2, d3, d4 = data_train.shape
    #data_train = data_train.reshape((d1, d2*d3*d4))
    d1, d2, d3, d4 = data_test.shape
    data_test = data_test.reshape((d1, d2*d3*d4))
    #data_train = torch.flatten(torch.from_numpy(data_train), start_dim=1)
    #data_test = torch.flatten(torch.from_numpy(data_test), start_dim=1)

    #data_train = data_train.cpu().detach().numpy()
    #data_test = data_test.cpu().detach().numpy()
    #exits_ground_truth_train = np.array(exits_ground_truth_train)
    exits_ground_truth_test = np.array(exits_ground_truth_test)
    #ipdb.set_trace(context=6)
    ros_over = RandomOverSampler(random_state=42)
    ros_under = RandomUnderSampler(random_state=42)
    sm = SMOTE(random_state=42)
    
    #data_train, exits_ground_truth_train = ros_over.fit_resample(data_train, exits_ground_truth_train)
    #data_test, exits_ground_truth_test = ros_over.fit_resample(data_test, exits_ground_truth_test)
    #data_train, exits_ground_truth_train = sm.fit_resample(data_train, exits_ground_truth_train)
    print('Resampled test dataset shape before %s' % Counter(exits_ground_truth_test))

    data_test, exits_ground_truth_test = sm.fit_resample(data_test, exits_ground_truth_test)
    #print('Resampled train dataset shape %s' % Counter(exits_ground_truth_train))
    print('Resampled test dataset shape %s' % Counter(exits_ground_truth_test))

    X_train, X_test, y_train, y_test = train_test_split(data_test, exits_ground_truth_test, test_size=0.33, random_state=42)
    data_test = None
    exits_ground_truth_test = None
    
    #ipdb.set_trace(context=6)
    
    #xgb_train = xgb.DMatrix(X_train, label=y_train)#, weight=weights_vec_train)
    #xgb_val = xgb.DMatrix(X_test, label=y_test)#, weight=weights_vec_test)
    #watchlist = [(xgb_train, 'train'), (xgb_val, 'val')]
    
    num_rounds = 1500
    #depth = 4
    # Training phase
    print("building model...")
    XGB = XGBRegressor(tree_method = "gpu_hist", objective = "multi:softmax", num_class=11,\
                       single_precision_histogram=True, max_depth=args.max_depth, eval_metric=["merror"],\
                       n_estimators=num_rounds, verbosity=1)
    #print(XGB)
    eva_set = [(X_test, y_test)]
    X_test = None
    y_test = None
    #ipdb.set_trace(context=6)
    
    #XGB = XGB.fit(X_train, y_train, eval_metric="auc", early_stopping_rounds=100, verbose=True)
    XGB = XGB.fit(X_train, y_train, verbose=True, early_stopping_rounds=5, eval_set=eva_set)
    #score = XGB.score(X_train, y_train)  
    #print("Training score: ", score)
    #plst = list(params.items())
    #gbm = xgb.train(plst, xgb_train, num_rounds, watchlist,early_stopping_rounds=100)
    print("saving model...")
    log_file.write("saving model...\n")
    XGB.save_model(args.gated_models_dir + '/xg_model.' + str(args.loss_threshold) + '.depth' + str(args.max_depth) + '.json')
    #jobl.dump(gbm, args.gated_models_dir + '/xg_model.' + str(args.loss_threshold) + '.bin')
    #validation_accuracy = evaluate(gbm, xgb_val, y_test)#, weights_vec_test)
    #validation_accuracy = evaluate(XGB, X_test, y_test)#, weights_vec_test)
    
    #print("  - validation accuracy = {0:.4f}".format(validation_accuracy))
    #log_file.write("validation accuracy: " + str(validation_accuracy) + "\n")
    log_file.close()
    return
    #return validation_accuracy

In [ ]:
#@title train_gating_model
from gatednet import gnet, gnet_s, gnet_l
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def train_gating_model(args, main_model, train_loader, test_loader, trainset):    
    
    gt_file_train = 'gated_ground_truths/exit_idx_predictions_train.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_train = torch.load(gt_file_train)
    print('loaded exits ground truths at: ', gt_file_train)
    train_len = len(exits_ground_truth_train)
    print('exits_ground_truth_train {:8d}'.format(train_len))
    
    gt_file_test = 'gated_ground_truths/exit_idx_predictions_test.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_test = torch.load(gt_file_test)
    print('loaded exits ground truths at: ', gt_file_test)
    test_len = len(exits_ground_truth_test)
    print('exits_ground_truth_test {:8d}'.format(test_len))
    
    data_file_train = 'gated_ground_truths/datas_train.' + str(args.loss_threshold) + '.pt'
    data_train = torch.load(data_file_train)
    print('loaded data train at: ', data_file_train)
    data_train_len = len(data_train)
    print('data_train_len {:8d}'.format(data_train_len))
    
    data_file_test = 'gated_ground_truths/datas_test.' + str(args.loss_threshold) + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    data_test_len = len(data_test)
    print('data_test_len {:8d}'.format(data_test_len))
    
    model = gnet((3, 32, 32), args.num_ee + 1, args.filters)
    #model = gnet_s((3, 32, 32), args.num_ee + 1, args.filters)
    #model = gnet_l((3, 32, 32), args.num_ee + 1)
    main_model.eval()
    main_model.set_full_flow(True)

    #args.device = 'cpu'
    model = model.to(args.device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    #optimizer = optim.Adam(model.parameters(), lr=args.lr)
    criterion = nn.CrossEntropyLoss()

    exits_ground_truth_train = torch.LongTensor(exits_ground_truth_train)
    #exit_gt_batches_train = list(chunks(exits_ground_truth_train, args.batch_size))
    exits_ground_truth_test = torch.LongTensor(exits_ground_truth_test)
    #exit_gt_batches_test = list(chunks(exits_ground_truth_test, args.batch_size))
    
    #print(data_train[0].shape)
    #data_train = torch.Tensor(data_train)
    data_train = torch.stack(data_train, dim=0)
    data_test = torch.stack(data_test, dim=0)
    print(data_train.shape)
    
    dataset_train = TensorDataset(data_train, exits_ground_truth_train)
    loader_train_new = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True)
    dataset_test = TensorDataset(data_test, exits_ground_truth_test)
    loader_test_new = DataLoader(dataset_test, batch_size=args.batch_size, shuffle=True)
            
    print('Running for {:5d} epochs'.format(args.epochs))
    try:
        if not args.no_tensorboard:
            writer = SummaryWriter('../runs/eenet_experiment_1')
        torch.cuda.empty_cache()
        for epoch in range(args.start_epoch, args.epochs + 1):
            print('{:3d}:'.format(epoch), end='')
            losses = []
            #i = 0
            model.train()
            for batch_id, (data, target) in enumerate(loader_train_new):
                data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
                #if batch_id * args.batch_size > train_len:
                #    break
                optimizer.zero_grad()
                pred = model(data)
                #ipdb.set_trace(context=6)   
                #loss = criterion(pred, target)
                loss = F.nll_loss(pred.log(), target)
                loss.backward()
                optimizer.step()

                losses.append(float(loss))
                #else:
                #    print('here')
                
                  
            print('Train avg loss: {:.4f}'.format(round(np.mean(losses), 4)))
            model.eval()
            #print('here')
            accs = []
            correct = 0
            batch_id = 0
            with torch.no_grad():
                #print('here1')
            
                for batch_id, (data, target) in enumerate(loader_test_new):
                    data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
                    #if batch_id * args.batch_size > test_len:
                    #    break
                    #labels = exit_gt_batches_test[batch_id].to(args.device)
                    
                    pred = model(data)
                    main_preds, _, _, _ = main_model(data)
                    #ipdb.set_trace(context=6) 
                    _, predicted = torch.max(pred.data, 1)
                    #_, main_predicted = torch.max(main_preds.data, 1)
                    
                    #correct += (predicted == target).sum().item()
                    pred = pred.max(1, keepdim=True)[1]
                    acc = pred.eq(target.view_as(pred)).sum().item()
                    accs.append(acc*100.)
            
            #print('Avg test acc is:, %d %%' % (100 * correct / test_len))
            print('Avg test acc is:, %d %%' % (round(np.mean(accs), 4) / test_len))

            

    except KeyboardInterrupt:
        sys.exit()
    
    filename = 'gated_models/' + str(args.dataset) + '/model.' + str(args.loss_threshold) + '.pt'
    torch.save(model, filename)

In [7]:
#@title load_generated_data

class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        #ipdb.set_trace(context=6)
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]
        z = self.tensors[2][index]
        to = self.tensors[3][index]
        
        return x, y, z, to

    def __len__(self):
        return self.tensors[0].size(0)
        
def load_generated_data(args, shuffle):
    target_file_name = 'targets_main_test'
    
    ground_truth_noisy_dir = args.ground_truths_dir + '/' + str(args.loss_threshold)# + '/' + 'snr_' + str(noise_levels[0])
    
    data_file_test = ground_truth_noisy_dir + '/datas_test.' + str(args.loss_threshold) + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    
    target_file_test = ground_truth_noisy_dir + '/' + target_file_name + '.' + str(args.loss_threshold) + '.pt'
    target_test = torch.load(target_file_test)
    print('loaded data test at: ', target_file_test)
    
    gt_file_test = ground_truth_noisy_dir + '/exit_idx_predictions_test.' + str(args.loss_threshold) + '.pt'
    exits_ground_truth_test = torch.load(gt_file_test)
    print('loaded exits ground truths test at: ', gt_file_test)
    
    levels_file_test = ground_truth_noisy_dir + '/levels_test.' + str(args.loss_threshold) + '.pt'
    levels_test = torch.load(levels_file_test)
    #ipdb.set_trace(context=6)
    print('levels test at: ', levels_file_test)

    validation_set = CustomTensorDataset(tensors=(torch.from_numpy(data_test), torch.from_numpy(exits_ground_truth_test), 
                                                  torch.from_numpy(levels_test), torch.from_numpy(target_test)))
    loader = torch.utils.data.DataLoader(validation_set, batch_size=args.test_batch, shuffle=shuffle)

    return validation_set, loader

NameError: ignored

In [ ]:
#@title generate_relative_loss
import xgboost as xgb
import joblib as jobl

def generate_relative_loss(args, main_model, noise_levels):
    losses = []
    acum_costs = []
    noise_exits = []
    #vals_len_true = np.zeros(args.num_ee+1)
    for i in range(args.num_noise_levels):
        noise_exits.append([])

    validation_set, loader = load_generated_data(args, shuffle=True)
    
    log_file = open(args.relative_losses_dir+'/relative_losses_log.txt', 'a', newline='')
    log_file.write("Generate relative losses:\n")
    log_file.write("loss threshold: " + str(args.loss_threshold) + ", SNR=" + str(args.noise_snr) + "\n")
    
    type_loader = 'test'
        
    #ipdb.set_trace(context=6)
    #args.device = 'cpu'
    #main_model.to(args.device)

    filename = args.gated_models_dir + '/xg_model.' + str(args.loss_threshold) + '.depth' + str(args.max_depth) + '.json'
    #gated_model = jobl.load(filename)
    gated_model = xgb.XGBRegressor()
    gated_model.load_model(filename)
    print('loaded gated model: ', filename)
    main_model.eval()
    main_model.set_full_flow(True)
    total_flops, _ = main_model.complexity[-1]
    #ipdb.set_trace(context=6)
    target_match_count = 0
    
    #it = 0
    with torch.no_grad():
        for data, target, noise, labels in loader:
            data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
            labels = labels.to(args.device, dtype=torch.int64)  
            noise = noise.to(args.device, dtype=torch.int64)  
            preds, _, _, _ = main_model(data)
            
            data = torch.flatten(data, start_dim=1)
            data_xgb = data.cpu().detach().numpy()
            target_xgb = target.cpu().detach().numpy()
            xgb_data = xgb.DMatrix(data_xgb, label=target_xgb)
            gated_pred = gated_model.predict(data_xgb).astype(int)
            #ipdb.set_trace(context=6)
            
            new_preds = torch.empty((args.test_batch, args.num_ee), device = args.device)
            gated_cost = np.zeros(args.test_batch)
            for i in range(args.test_batch):
                #ipdb.set_trace(context=6)
                new_preds[i] = preds[gated_pred[i]][i]
                gated_cost[i] = main_model.complexity[gated_pred[i]][0]/total_flops
                for j in range(args.num_noise_levels):
                    if noise_levels[j] == noise[i]:
                        #ipdb.set_trace(context=6)
                        noise_exits[j].append(gated_pred[i])
            
            pred_targets = torch.max(new_preds, 1).indices
            _, target_main = torch.max(preds[args.num_ee], 1)
                
            #cum_loss = F.nll_loss(preds[gated_pred].log(), target)
            cum_loss = F.nll_loss(new_preds.log(), target_main)
            losses.append(float(cum_loss))
            acum_costs.append(gated_cost)
            target_match_count = target_match_count + torch.sum(pred_targets==labels)
            #ipdb.set_trace(context=6)
            
            #print('batch idx = ', it)
            #it = it + 1
        
        mean_rel_loss = round(np.mean(losses), 4)
        print('Mean relative loss: {:.4f}'.format(mean_rel_loss))
        log_file.write("Mean relative loss: " + str(mean_rel_loss) + "\n")
        mean_rel_cost = round(np.mean(acum_costs), 4)
        print('Mean relative cost: {:.4f}'.format(mean_rel_cost))
        log_file.write("Mean relative cost: " + str(mean_rel_cost) + "\n")
        accu = target_match_count.cpu().detach()/len(validation_set)
        log_file.write("Accuracy: " + str(accu) + "\n")
        print('Accuracy: {:.4f}'.format(float(accu)))
        torch.save(noise_exits, args.relative_losses_dir + '/noise_exits_' + type_loader + '.' + str(args.loss_threshold) + '.pt')
        
        #plot_noise_exits(args, noise_exits, args.relative_losses_dir)
        return mean_rel_loss, mean_rel_cost, accu
            

In [ ]:
#@title calc_relative_time
import xgboost as xgb
import joblib as jobl
import json

def calc_relative_time(args, main_model, noise_levels):
    losses = []
    acum_costs = []
    noise_exits = []
    #vals_len_true = np.zeros(args.num_ee+1)
    for i in range(args.num_noise_levels):
        noise_exits.append([])
    
    log_file = open(args.relative_losses_dir+'/calc_relative_time.txt', 'a', newline='')
    log_file.write("Calc relative time:\n")
    log_file.write("loss threshold: " + str(args.loss_threshold) + ", SNR=" + str(args.noise_snr) + "\n")
    type_loader = 'test'
        
    validation_set, loader = load_generated_data(args, shuffle=False)
    
    #ipdb.set_trace(context=6)
    #args.device = 'cpu'
    #main_model.to(args.device)

    #filename = args.gated_models_dir + '/xg_model.' + str(args.loss_threshold) + '.bin'
    filename = args.gated_models_dir + '/xg_model.' + str(args.loss_threshold) + '.depth' + str(args.max_depth) + '.json'
    #gated_model = jobl.load(filename)
    gated_model = xgb.XGBRegressor()
    gated_model.load_model(filename)

    #gated_model = xgb.Booster()
    #gated_model.load_model(filename)
    print('loaded gated model: ', filename)
    #gated_model.set_param({"predictor": "gpu_predictor"})
    #gated_model.set_param({"tree_method": "hist"})
    #gated_model.set_param({"single_precision_histogram": True})
    main_model.eval()
    main_model.set_full_flow(True)
    total_flops, _ = main_model.complexity[-1]
    #ipdb.set_trace(context=6)
    target_match_count = 0

    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    
    #it = 0
    with torch.no_grad():
        for data, target, noise, labels in loader:
            data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
            labels = labels.to(args.device, dtype=torch.int64)  
            noise = noise.to(args.device, dtype=torch.int64)
            #dummy_input = torch.randn(1, data.shape[1],dtype=torch.float).to(args.device)

            
            starter.record()  
            preds, _, costs, _ = main_model(data)
            ender.record()
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            print('avg time main:', curr_time/args.test_batch)
            
            data = torch.flatten(data, start_dim=1)
            data_xgb = data.cpu().detach().numpy()
            target_xgb = target.cpu().detach().numpy()
            xgb_data = xgb.DMatrix(data_xgb, label=target_xgb)

            #ipdb.set_trace(context=6)  
            #GPU-WARM-UP
            #for _ in range(10):
            #    _,_,_,_ = main_model(dummy_input)    
            
            starter.record()
            gated_pred = gated_model.predict(data_xgb)
            ender.record()
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            print('avg time xg:', curr_time/args.test_batch)
            
            #ipdb.set_trace(context=6)
            """
            total_cost = 0
            
            gated_pred = gated_pred.astype(int)
            new_preds = torch.empty((args.test_batch, args.num_ee), device = args.device)
            #gated_cost = np.zeros(args.test_batch)
            for i in range(args.test_batch):
                ipdb.set_trace(context=6)
                total_cost = total_cost + costs[gated_pred[i]][i]
                new_preds[i] = preds[gated_pred[i]][i]
                #gated_cost[i] = main_model.complexity[gated_pred[i]][0]/total_flops
                #for j in range(args.num_noise_levels):
                #    if noise_levels[j] == noise[i]:
                #        #ipdb.set_trace(context=6)
                #        noise_exits[j].append(gated_pred[i])
            
            pred_targets = torch.max(new_preds, 1).indices
            _, target_main = torch.max(preds[args.num_ee], 1)
            
            
            avg_cost = total_cost/args.test_batch 
            print("avg cost: ", avg_cost)  
            target_match_count = target_match_count + torch.sum(pred_targets==pred_targets)
            accu = target_match_count.cpu().detach()/len(validation_set)
            log_file.write("Accuracy: " + str(accu) + "\n")
            print('Accuracy: {:.4f}'.format(float(accu)))
            """
            """
            #cum_loss = F.nll_loss(preds[gated_pred].log(), target)
            cum_loss = F.nll_loss(new_preds.log(), target_main)
            losses.append(float(cum_loss))
            acum_costs.append(gated_cost)
            target_match_count = target_match_count + torch.sum(pred_targets==labels)
            #ipdb.set_trace(context=6)
            """
            #print('batch idx = ', it)
            #it = it + 1
        """
        mean_rel_loss = round(np.mean(losses), 4)
        print('Mean relative loss: {:.4f}'.format(mean_rel_loss))
        log_file.write("Mean relative loss: " + str(mean_rel_loss) + "\n")
        mean_rel_cost = round(np.mean(acum_costs), 4)
        print('Mean relative cost: {:.4f}'.format(mean_rel_cost))
        log_file.write("Mean relative cost: " + str(mean_rel_cost) + "\n")
        """
        return
            

In [ ]:
#@title Validate
def validate(args, model, val_loader):
    """validate the model.

    Arguments are
    * args:         command line arguments entered by user.
    * model:        convolutional neural network model.
    * val_loader:   validation data loader..

    This validates the model and prints the results of each epochs.
    Finally, it returns average accuracy, loss and comptational cost.
    """
    batch = {'time':[], 'cost':[], 'flop':[], 'acc':[], 'val_loss':[]}
    exit_points = [0]*(args.num_ee+1)
    # switch to evaluate mode
    model.eval()
    if args.loss_threshold != 0:
        model.set_full_flow(True)
    with torch.no_grad():
        #for data, target in val_loader:
        for data, target, _ in val_loader:
            #if args.add_noise != 0:
            #    data = add_noise(args, data)
        
            data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
            # compute output
            start = time.process_time()

            # results of EENet based models
            if isinstance(model, (EENet, CustomEENet)):                
                pred, conf, cost, idx = model(data)
                elapsed_time = time.process_time() - start
                #loss = F.cross_entropy(pred, target)
                if args.loss_threshold != 0:
                    idx = args.num_ee
                    for i in range(args.num_ee):
                        pred_loss = F.nll_loss(pred[i].log(), target)
                        #print(pred_loss)
                        #return
                        if pred_loss < args.loss_threshold:
                            idx = i
                            #print('target:' + str(target) + ', ee:' + str(i))
                            break
                    pred = pred[idx]
                    cost = cost[idx]
                loss = F.nll_loss(pred.log(), target) + args.lambda_coef * cost
                flop, cost_ = model.complexity[-1][0], 1.0
                exit_points[idx] += 1

            # results of other models
            else:
                pred = model(data)
                elapsed_time = time.process_time() - start
                loss = F.cross_entropy(pred, target)
                flop, cost = model.complexity[-1][0], 1.0
                exit_points = None

            # get the index of the max log-probability
            pred = pred.max(1, keepdim=True)[1]
            acc = pred.eq(target.view_as(pred)).sum().item()
            batch['acc'].append(acc*100.)
            batch['time'].append(elapsed_time)
            batch['cost'].append(cost*100.)
            batch['flop'].append(flop)
            batch['val_loss'].append(float(loss))
            
    utils.print_validation(args, batch, exit_points)
    model.set_full_flow(False)

    result = {}
    for key, value in batch.items():
        result[key] = round(np.mean(value), 4)
        result[key+'_sem'] = round(stats.sem(value), 2)
    
    result['exit_points'] = exit_points
    return result


In [ ]:
#@title generate_ee_exit_prediction_ground_truths
def generate_ee_exit_prediction_ground_truths(args, model, train_loader, test_loader, noise_levels):
    
    model.eval()
    model.set_full_flow(True)
    print('shuffle train: ', args.shuffle_train)
    
    #for m in range(args.num_noise_levels):
    type_loader = 'train'
    loader = train_loader
    for k in range(2):
        first = True
        if k == 1:
            loader = test_loader
            type_loader = 'test'
        exit_idx_predictions = []
        noise_idxs = []
        datas = []
        #targets_org = []
        targets_main = []
        noise_lvls = []
        loss_main = []
        val_confs_true = []
        vals_len_true = np.zeros(args.num_ee+1)
        for i in range(args.num_ee+1):
            val_confs_true.append([])
        
        print('processing loader type: ', type_loader)
        print('batch size: {:d}'.format(args.batch_size))
        #print('noise level index : {:d}'.format(m))

        with torch.no_grad():
            for data, _, levels in loader:
                #if args.add_noise != 0:
                #    args.noise_snr = noise_levels[m]
                #    data = add_noise(args, data)
                #ipdb.set_trace(context=6)
                #data, target_org = data.to(args.device), target_org.to(args.device, dtype=torch.int64)
                data = data.to(args.device)
                preds, _, _, _ = model(data)
                
                if args.use_main_targets:
                    _, target_main = torch.max(preds[args.num_ee], 1)
                
                #ipdb.set_trace(context=6)
                best_exit = np.full(target_main.shape[0], args.num_ee, dtype=int) #args.num_ee
                for j in range(target_main.shape[0]):
                    for i in range(args.num_ee):
                        pred = preds[i][j][None, :]
                        pred_loss = F.nll_loss(pred.log(), target_main[j].unsqueeze(0))
                    
                        if pred_loss < args.loss_threshold:
                            best_exit[j] = i
                            break

                    val_confs_true[best_exit[j]].append(pred_loss)

                #ipdb.set_trace(context=6)
                exit_idx_predictions = np.append(exit_idx_predictions, best_exit)
                
                if first:
                    first = False
                    datas = data.cpu().detach().numpy()
                else:
                    datas = np.append(datas, data.cpu().detach().numpy(), axis=0)
                
                noise_lvls = np.append(noise_lvls, levels.cpu().detach().numpy())
                #targets_org = np.append(targets_org, target_org.cpu().detach().numpy())
                targets_main = np.append(targets_main, target_main.cpu().detach().numpy())
                #noise_idxs = np.append(noise_idxs, noise_level_idx.cpu().detach().numpy())
                loss_main.append(float(F.nll_loss(preds[args.num_ee].log(), target_main)))
                
        print('exit_idx_predictions length {:8d}'.format(len(exit_idx_predictions)))
        #print('noise_idxs length {:8d}'.format(len(noise_idxs)))
        print('loss_main mean: {:.2f}'.format(round(np.mean(loss_main), 4)))
        
        ground_truth_noisy_dir = args.ground_truths_dir + '/' + str(args.loss_threshold)# + '/' + 'snr_' + str(noise_levels[m])
        if not os.path.exists(ground_truth_noisy_dir):
            os.makedirs(ground_truth_noisy_dir)

        torch.save(exit_idx_predictions, ground_truth_noisy_dir
                + '/exit_idx_predictions_' + type_loader + '.' + str(args.loss_threshold) + '.pt')
        #torch.save(noise_idxs, ground_truth_noisy_dir
        #        + '/noise_idxs_' + type_loader + '.' + str(args.loss_threshold) + '.pt')
        
        if type_loader == 'test':
            torch.save(datas, ground_truth_noisy_dir + '/datas_' + type_loader + '.' + str(args.loss_threshold) + '.pt')
        torch.save(targets_main, ground_truth_noisy_dir + '/targets_main_' + type_loader + '.' + str(args.loss_threshold) + '.pt')
        #torch.save(targets_org, ground_truth_noisy_dir + '/targets_org_' + type_loader + '.' + str(args.loss_threshold) + '.pt')
        torch.save(noise_lvls, ground_truth_noisy_dir + '/levels_' + type_loader + '.' + str(args.loss_threshold) + '.pt')
        for i in range(args.num_ee+1):
            vals_len_true[i] = len(val_confs_true[i])
        plot_ee_histogram(args, vals_len_true, type_loader, ground_truth_noisy_dir)
        

In [ ]:
#@title run()
def run(model, optimizer, args, train_loader, test_loader, exit_tags):
    # best = {}
    # best_epoch = 0
    
    print('Running for {:5d} epochs'.format(args.epochs))
    try:
        if not args.no_tensorboard:
            writer = SummaryWriter('../runs/eenet_experiment_1')
        torch.cuda.empty_cache()
        for epoch in range(args.start_epoch, args.epochs + 1):
            print('{:3d}:'.format(epoch), end='')

            # two-stage training uses the loss version-1 after training for 25 epochs
            if args.two_stage and epoch > 25:
                args.loss_func = "v1"
            
            # use adaptive learning rate
            if args.adaptive_lr:
                utils.adaptive_learning_rate(args, optimizer, epoch)

            result = {'epoch':epoch}
            #args.noise_snr = noise_levels[epoch%args.num_noise_levels]
            #print('noise SNR = ', args.noise_snr)
            
            result.update(train(args, model, train_loader, optimizer, exit_tags))
            if not args.no_tensorboard:
                writer.add_scalar("Train/loss", result['train_loss'], epoch)

            # validate and keep history at each log interval
            if epoch % args.log_interval == 0:
                result.update(validate(args, model, test_loader))
                utils.save_history(args, result)
                if not args.no_tensorboard:
                    writer.add_scalar("Val/loss", result['val_loss'], epoch)
                    writer.add_scalars('Val/acc_cost', {
                        'acc': result['acc'],
                        'cost': result['cost']
                    }, epoch)
            if not args.no_tensorboard:    
                writer.flush()
            # save model parameters
            if not args.no_save_model:
                utils.save_model(args, model, epoch)
    except KeyboardInterrupt:
        utils.close_history(args)
        utils.plot_history(args)
        if not args.no_tensorboard:
            writer.close()
        sys.exit()
    # print the best validation result
    best_epoch = utils.close_history(args)

    # save the model giving the best validation results as a final model
    if not args.no_save_model:
        utils.save_model(args, model, best_epoch, True)
    utils.plot_history(args)
    if not args.no_tensorboard:
        writer.close()


In [ ]:
#@title enable_branches_training_only
def enable_branches_training_only(model, args):
    if args.model == 'eenet8':
        model.set_ee_disable(False)
        model.initblock.requires_grad_(False)
        model.basicblock1.requires_grad_(False)
        model.basicblock2.requires_grad_(False)
        model.basicblock3.requires_grad_(False)
        model.finalblock.requires_grad_(False)
        model.classifier.requires_grad_(False)
        model.conv2d_6.requires_grad_(False)
        model.conv2d_9.requires_grad_(False)
    else:
        model.set_ee_disable(False)
        for idx, exitblock in enumerate(model.exits):
            model.stages[idx].requires_grad_(False)
            for param in model.exits.parameters():
                param.requires_grad = True
            #model.exits.requires_grad(True)

        model.stages[-1].requires_grad_(False)
        model.classifier.requires_grad_(False)
        model.confidence.requires_grad_(False)
        #params = model_post.state_dict()
        #print(params)
        """
        for name, p in model.named_parameters():
            if p.requires_grad:
                print(name, p.requires_grad)
        """

In [ ]:
#@title plot_relative_loss
def plot_relative_loss():
    rel_losses_no_noise = np.array([0.2629, 0.2704, 0.2939, 0.3123, 0.3423, 0.3687, 0.3952, 0.4234, 0.4565,
        0.4912, 0.519])
    rel_losses_20db = np.array([0.2847, 0.3419, 0.3947, 0.4448, 0.4891, 0.523,  0.5664, 
        0.615,  0.6568, 0.6988, 0.7474])
    rel_losses_10db = np.array([0.2886, 0.3495, 0.3812, 0.4199, 0.4694, 0.5304, 0.5900, 0.6455,
        0.7120, 0.7603, 0.8047])
    rel_losses_30db = np.array([0.2496, 0.3067, 0.3593, 0.4223, 0.4915, 0.5604, 0.5943, 0.6343, 
        0.6828, 0.728, 0.7757])
    
    
    #print('rel_losses: ', rel_losses)
    main_loss_no_noise = 0.12
    main_loss_20db = 0.14
    main_loss_30db = 0.14
    main_loss_10db = 0.14

    x = np.arange(0.2, 1.3, 0.1)
    
    y_no_noise = rel_losses_no_noise/main_loss_no_noise
    x_no_noise = x/main_loss_no_noise
    y_20db = rel_losses_20db/main_loss_20db
    x_20db = x/main_loss_20db
    y_10db = rel_losses_10db/main_loss_10db
    x_10db = x/main_loss_10db
    y_30db = rel_losses_30db/main_loss_30db
    x_30db = x/main_loss_30db
    
    %matplotlib inline
    plt.title("Test loss vs loss threshold increase")
    plt.xlabel("loss threshold")
    plt.ylabel("test loss")
    plt.xlim(0, max(x_no_noise)+1)
    plt.ylim(0, max(y_no_noise)+3)
    xpoints = ypoints = plt.xlim()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.plot(xpoints, ypoints, linestyle='--', color='k', lw=3, scalex=False, scaley=False)
    plt.plot(x_no_noise, y_no_noise, color ="red", marker='o')
    plt.plot(x_20db, y_20db, color ="blue", marker='x')
    plt.plot(x_10db, y_10db, color ="green", marker='x')
    plt.plot(x_30db, y_30db, color ="cyan", marker='x')
    plt.show()

In [ ]:
#@title load data
def load_data(args):
    if args.add_noise == 0:
        print('load plain dataset')
        return utils.load_dataset(args)
    else:
        print('load noisey dataset')
        return utils.load_noisy_dataset(args)

In [ ]:
#@title main

"""Main function of the program.

The function loads the dataset and calls training and validation functions.
"""
%load_ext autoreload
%autoreload 2

import importlib
importlib.reload(config)
importlib.reload(loss_functions)
importlib.reload(utils)
#from eenet import EENet
#from gatednet import GatedNet, GatedNetS
from enum import Enum

class Mode(Enum):
    train_main = 0
    train_ee = 1
    generate_exits_gt = 2 
    train_gating = 3
    generate_relative_loss = 4 
    plot_relative_loss = 5
    calc_relative_time = 6

def main(mode: Mode):
    print(mode.value)
    args = config.args_global
    args += config.argu[mode.value]
    
    print(args)
    %pwd
    model, optimizer, args = initializer(args)
    noise_levels = np.arange(args.min_noise_snr, args.max_noise_snr,
            (args.max_noise_snr - args.min_noise_snr)/args.num_noise_levels) 
    print(noise_levels)

    if mode == Mode.train_main:
        train_loader, test_loader, exit_tags, trainset, testset = load_data(args)
        model.set_ee_disable(True)
        print('Disabled EE branches')
        run(model, optimizer, args, train_loader, test_loader, exit_tags)
    elif mode == Mode.train_ee:
        train_loader, test_loader, exit_tags, trainset, testset = load_data(args)
        enable_branches_training_only(model, args)
        print('Enabled EE branches')
        run(model, optimizer, args, train_loader, test_loader, exit_tags)
    elif mode == Mode.generate_exits_gt:
        train_loader, test_loader, exit_tags, trainset, testset = load_data(args)       
        for thresh in range(1, 13, 1):
            args.loss_threshold = thresh/10
            print('Generating ground truth for ee loss, threshold = {:.2f}'.format(args.loss_threshold))
            generate_ee_exit_prediction_ground_truths(args, model, train_loader, test_loader, noise_levels)
    #elif config.mode == 'analyze_ee_data_flow':
    #    exit_flow_analysis(args, model, test_loader)
    elif mode == Mode.train_gating:
        print('Training Gated model')
        accus = []
        for thresh in range(7, 8, 1):
            args.loss_threshold = thresh/10
            val_accu = train_gated_xgboost(args, noise_levels)
            accus.append(val_accu)
        print('val_accuracies: ', np.array(accus))
        #train_gating_model(args, model, train_loader, test_loader, trainset)
    elif mode == Mode.generate_relative_loss:
        print('Generate relative loss')
        rel_losses = []
        costs = []
        accus = []
        main_loss = 0.14
        for thresh in range(7, 8, 1):
            args.loss_threshold = thresh/10
            rel_loss, cost, accu = generate_relative_loss(args, model, noise_levels)
            rel_losses.append(rel_loss)
            costs.append(cost)
            accus.append(accu)
        #main_loss = 0.14
        print('rel_losses: ', np.array(rel_losses))
        print('costs: ', np.array(costs))
        print('accus: ', np.array(accus))
        #plot_relative_loss(np.array(rel_losses), main_loss)
    elif mode == Mode.plot_relative_loss:
        print('Plot relative loss')
        dataset_type = 'test'
        plot_noise_exits_scatter(args, dataset_type)
    elif mode == Mode.calc_relative_time:
        print('Calc relative time')
        for thresh in range(7, 8, 1):
            args.loss_threshold = thresh/10
            calc_relative_time(args, model, noise_levels)

    print('Finished')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
mode = Mode.calc_relative_time
main(mode)

6
['--dataset', 'cifar10', '--model', 'eenet110', '--num-ee', '10', '--filters', '4', '--exit-type', 'conv2', '--distribution', 'fine', '--add-noise', '--min-noise-snr', '-5', '--max-noise-snr', '30', '--num-noise-levels', '5', '--test-batch', '10000', '--load-model', 'models/cifar10/eenet110/UT/full_noisy/after_ee_training/model.pt', '--max-depth', '10']
use cuda:  True  device:  cuda
empty model loaded at:  models/cifar10/eenet110/UT/full_noisy/after_ee_training/model.pt
ee-block-0: flops=19.78 MMac, params=19.34 k, cost-rate=0.08
ee-block-1: flops=34.23 MMac, params=33.35 k, cost-rate=0.13
ee-block-2: flops=43.86 MMac, params=42.7 k, cost-rate=0.17
ee-block-3: flops=53.49 MMac, params=52.04 k, cost-rate=0.21
ee-block-4: flops=63.13 MMac, params=61.39 k, cost-rate=0.25
ee-block-5: flops=72.76 MMac, params=70.73 k, cost-rate=0.28
ee-block-6: flops=82.4 MMac, params=80.08 k, cost-rate=0.32
ee-block-7: flops=92.03 MMac, params=89.42 k, cost-rate=0.36
ee-block-8: flops=100.48 MMac, param